In [1]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torchvision.models import ResNet50_Weights
import numpy as np
from torch.optim.swa_utils import AveragedModel, SWALR
from tqdm import tqdm

# Load the train.csv file
df = pd.read_csv('train.csv')

# Define paths
train_img_path = 'train/train'
output_path = 'prepared_data'

# Split dataset into train and validation sets (80% train, 20% val)
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Mapping for jenis to human-readable labels
jenis_dict = {0: 'kaos', 1: 'hoodie'}

class MultilabelDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_id = self.dataframe.iloc[idx]['id']
        img_path_jpg = os.path.join(self.img_dir, f'{img_id}.jpg')
        img_path_png = os.path.join(self.img_dir, f'{img_id}.png')
        
        if os.path.exists(img_path_jpg):
            img_path = img_path_jpg
        elif os.path.exists(img_path_png):
            img_path = img_path_png
        else:
            return None

        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        jenis_label = self.dataframe.iloc[idx]['jenis']
        label = torch.tensor(jenis_label)

        return image, label

# Define transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Dataset and DataLoader setup
train_dataset = MultilabelDataset(train_df, train_img_path, transform=data_transforms['train'])
val_dataset = MultilabelDataset(val_df, train_img_path, transform=data_transforms['val'])

# Custom collate function
def collate_fn(batch):
    batch = list(filter(lambda x: x is not None, batch))
    return torch.utils.data.dataloader.default_collate(batch)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0, pin_memory=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0, pin_memory=True, collate_fn=collate_fn)

dataloaders = {'train': train_loader, 'val': val_loader}
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Model Definition with MC Dropout
class CustomResNetWithMC(nn.Module):
    def __init__(self, num_classes_type=2):
        super(CustomResNetWithMC, self).__init__()
        self.model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Sequential(
            nn.Linear(num_ftrs, 256),
            nn.ReLU(),
            nn.Dropout(0.5),  # MC Dropout
            nn.Linear(256, num_classes_type)  # 2 for jenis (kaos and hoodie)
        )

    def forward(self, x):
        return self.model(x)

# Initialize model, criteria, optimizer, and scheduler
model = CustomResNetWithMC().to(device)
swa_model = AveragedModel(model).to(device)

# Optimizer and SWA scheduler
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = SWALR(optimizer, anneal_strategy='cos', anneal_epochs=5, swa_lr=0.0001)

# Class weights for loss
jenis_counts = train_df['jenis'].value_counts().sort_index().values
class_weights_jenis = 1. / torch.tensor(jenis_counts, dtype=torch.float).to(device)
criterion_jenis = nn.CrossEntropyLoss(weight=class_weights_jenis)

# Training function
def train_model(model, swa_model, optimizer, scheduler, num_epochs=25):
    best_model_wts = model.state_dict()
    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion_jenis(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    _, preds = torch.max(outputs, 1)
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += (preds == labels).sum().item()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print(f'{epoch}-{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'train':
                swa_model.update_parameters(model)
                scheduler.step()

        if epoch >= (num_epochs - 5):
            torch.optim.swa_utils.update_bn(dataloaders['train'], swa_model, device=device)

    model.load_state_dict(best_model_wts)
    return swa_model

# Training with SWA
model = train_model(model, swa_model, optimizer, scheduler, num_epochs=25)

# Testing function with MC Dropout for Bayesian inference
def bayesian_predict_mc_dropout(model, image, n_samples=50):
    model.eval()
    preds_jenis = []

    with torch.no_grad():
        for _ in range(n_samples):
            output = model(image)
            output_jenis = output[:, :2]
            preds_jenis.append(torch.softmax(output_jenis, dim=1).cpu().numpy())

    mean_jenis = np.mean(preds_jenis, axis=0)
    uncertainty_jenis = np.std(preds_jenis, axis=0)

    return mean_jenis, uncertainty_jenis

# Directory containing test images
test_dir = 'test/test'
supported_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')
test_images = [f for f in os.listdir(test_dir) if f.lower().endswith(supported_extensions)]

predictions = []
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Count predictions for each label for debugging purposes
jenis_counts = {}

for image_name in test_images:
    image_path = os.path.join(test_dir, image_name)
    img = Image.open(image_path).convert('RGB')
    img = transform(img).unsqueeze(0).to(device)
    mean_jenis, _ = bayesian_predict_mc_dropout(model, img)

    jenis_pred = np.argmax(mean_jenis, axis=1).item()

    # Add predictions to list
    predictions.append({'id': int(os.path.splitext(image_name)[0]), 'jenis': jenis_pred})

    # Update counts for debug
    jenis_counts[jenis_pred] = jenis_counts.get(jenis_pred, 0) + 1

# Print prediction distribution
print("Jenis Prediction Distribution:", jenis_counts)

# Save to Excel
submission_df = pd.DataFrame(predictions).sort_values(by='id')

0-train Loss: 0.3502 Acc: 0.8712
0-val Loss: 0.0446 Acc: 1.0000
1-train Loss: 0.0709 Acc: 0.9807
1-val Loss: 0.0472 Acc: 0.9808
2-train Loss: 0.0450 Acc: 0.9871
2-val Loss: 0.1159 Acc: 0.9679
3-train Loss: 0.0352 Acc: 0.9903
3-val Loss: 0.0695 Acc: 0.9744
4-train Loss: 0.0418 Acc: 0.9887
4-val Loss: 0.0383 Acc: 0.9744
5-train Loss: 0.0301 Acc: 0.9903
5-val Loss: 0.0679 Acc: 0.9808
6-train Loss: 0.0171 Acc: 0.9952
6-val Loss: 0.0530 Acc: 0.9872
7-train Loss: 0.0078 Acc: 0.9984
7-val Loss: 0.0554 Acc: 0.9744
8-train Loss: 0.0097 Acc: 0.9968
8-val Loss: 0.0547 Acc: 0.9808
9-train Loss: 0.0163 Acc: 0.9936
9-val Loss: 0.0398 Acc: 0.9744
10-train Loss: 0.0482 Acc: 0.9855
10-val Loss: 0.0668 Acc: 0.9808
11-train Loss: 0.0091 Acc: 0.9968
11-val Loss: 0.0679 Acc: 0.9615
12-train Loss: 0.0047 Acc: 0.9984
12-val Loss: 0.0801 Acc: 0.9744
13-train Loss: 0.0160 Acc: 0.9936
13-val Loss: 0.0341 Acc: 0.9872
14-train Loss: 0.0080 Acc: 0.9968
14-val Loss: 0.1172 Acc: 0.9744
15-train Loss: 0.0167 Acc: 0.9

PermissionError: [Errno 13] Permission denied: 'submission.xlsx'

In [2]:
submission_df.to_excel('submission.xlsx', index=False)
print("Predictions saved to submission.xlsx")


Predictions saved to submission.xlsx
